### Загрузка данных

Подключим pandas и загрузим файлы с данными

In [1]:
import pandas as pd

In [40]:
app_users = pd.read_csv('app_users.csv')
app_users.head()

,tracking_id,click_id,install_datetime,os_version,device_manufacturer,device_model,device_type,device_locale,revenue
0,1033522010023579020,22817403-1473193873-1098301704,2016-09-06 23:51:28,4.2.2,Lenovo,Lenovo A316i,phone,ru-RU,1.662699
1,1033522010023579020,22817403-1473195092-622681650,2016-09-07 00:11:19,5.0.2,Samsung,Galaxy A3,phone,ru-RU,1.863439
2,1033522010023579020,22817403-1473189457-258078900,2016-09-06 22:18:50,4.4.2,HTC,HTC Desire 601,phone,ru-RU,1.520538
3,1033522010023579020,22817403-1473192316-1183289285,2016-09-06 23:06:59,6.0,Huawei,CHM-U01,phone,ru-RU,1.781050
4,1033522010023579020,22817403-1473188854-1249048763,2016-09-06 22:20:11,4.4.2,Samsung,Galaxy S4 Mini,phone,ru-RU,1.669396


In [41]:
clicks = pd.read_csv('clicks.csv')
clicks.head()

,tracking_id,click_id,click_datetime,CPI
0,1033522010023579020,22817403-1473191644-1641538988,2016-09-06 22:54:04,0.786682
1,1033522010023579020,22817403-1473191680-501021762,2016-09-06 22:54:40,0.786682
2,1033522010023579020,22817403-1473193873-1098301704,2016-09-06 23:31:13,0.786682
3,1033522010023579020,22817403-1473195092-622681650,2016-09-06 23:51:32,0.786682
4,1033522010023579020,22817403-1473189457-258078900,2016-09-06 22:17:37,0.786682


Выведем сколько всего было различных компаний и увидим, что 6

In [26]:
print(sorted(clicks.tracking_id.unique()))
print(sorted(app_users.tracking_id.unique()))

[457095801521389451, 457100182639649595, 529165038158632239, 817383831807028900, 1033522010023579020, 1105628863007835516]
[457095801521389451, 457100182639649595, 529165038158632239, 817383831807028900, 1033522010023579020, 1105628863007835516]


Пронумеруем кампании для удобства

In [45]:
ref = {457095801521389451: 'c1', 
       457100182639649595: 'c2', 
       529165038158632239: 'c3', 
       817383831807028900: 'c4', 
       1033522010023579020: 'c5', 
       1105628863007835516: 'c6'}
clicks['campaign'] = clicks.tracking_id.map(lambda x: ref[x])
app_users['campaign'] = app_users.tracking_id.map(lambda x: ref[x])

Посчитаем охват и суммарные инсталлы для каждой из кампаний

In [71]:
coverage = clicks.groupby('campaign').click_id.count().sort_values()
installs = app_users.groupby('campaign').install_datetime.count().sort_values()

Рассчитаем конверсию

In [72]:
conv = pd.concat([coverage, installs], axis=1)
conv.columns = ['coverage', 'installs']
conv['conversion'] = conv.installs.div(conv.coverage)
conv.head(6)

,coverage,installs,conversion
campaign,,,
c6,695,81,0.116547
c3,4461,535,0.119928
c4,5106,554,0.108500
c1,7836,618,0.078867
c2,11918,1047,0.087850
c5,51681,6788,0.131344


Рассчитаем суммарную выручку и суммарные косты за клики

In [60]:
rev = app_users.groupby('campaign').revenue.sum()
cpi = clicks.groupby('campaign').CPI.sum()

In [74]:
df = pd.concat([conv, rev, cpi], axis=1)
df.columns = ['coverage', 'installs', 'conversion', 'total_revenue', 'total_cpi']

Рассчитаем ROI для каждой из кампаний

In [78]:
df['roi'] = df.total_revenue.div(df.total_cpi)
df.sort_values(by=['roi', 'conversion', 'coverage']).head(6)

,coverage,installs,conversion,total_revenue,total_cpi,roi
c1,7836,618,0.078867,962.088388,5143.797011,0.187039
c3,4461,535,0.119928,829.185564,3691.276970,0.224634
c5,51681,6788,0.131344,10516.624754,40656.499384,0.258670
c2,11918,1047,0.087850,1623.083851,6046.307733,0.268442
c4,5106,554,0.108500,855.027816,3137.636289,0.272507
c6,695,81,0.116547,123.641566,453.357454,0.272724


#### Выводы

Основными параметрами оценки маркетинговых кампаний можно считать: охват, конверсию, общую выручку и ROI.
По этим параметрам наиболее эффективной можно считать кампанию 5 (1033522010023579020). 
 * охват максимальный, на порядок больше, чем у остальных кампаний
 * конверсия также наибольшая из шести кампаний
 * ROI чуть меньше, чем у кампаний 2, 4, 6, но в пределах 1,5%
 * общая выручка тоже максимальна с учетом трех предыдущих пунктов